In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import mpl_utility_func 
import mpl_estimation

In [5]:
#mpl_utility_func.mpl_v(0,10,params=np.array([1,.8,10]),dstyle='attention')
#mpl_utility_func.mpl_v(0,10,args={'rra':1,'k':2},dstyle='hb')
mpl_utility_func.mpl_v(10,10,params={'rra':1,'beta':.5,'delta':.8},dstyle='quasihb')
#mpl_utility_func.mpl_v(0,10,args={'rra':1,'k':1,'a':1},dstyle='lp')
#mpl_utility_func.mpl_v(0,10,args={'rra':1,'k':1,'b':1},dstyle='bbs')
#mpl_utility_func.mpl_v(10,10,params={'rra':1,'beta':2},dstyle='gb')

array([0.53687091])

In [6]:
chavez_dt = pd.read_csv('data/chavez_data.csv')

# Kirby's questionnaire
pregunta = np.array([54,0,55,117,55,0,75,61,19,0,25,53,31,0,85,7,
                     14,0,25,19,47,0,50,160,15,0,35,13,25,0,60,14,
                     78,0,80,162,40,0,55,62,11,0,30,7,67,0,75,119,
                     34,0,35,186,27,0,50,21,69,0,85,91,49,0,60,89,
                     80,0,85,157,24,0,35,29,33,0,80,14,28,0,30,179,
                     34,0,50,30,25,0,30,80,41,0,75,20,54,0,60,111,
                     54,0,80,30,22,0,25,136,20,0,55,7]).reshape(27, 4)

condition_col = np.array(['DT'+str(i) for i in range(1, 28)]).reshape(-1,1)
pregunta = np.hstack((condition_col,pregunta))
pregunta = pd.DataFrame(pregunta)
condition_col_name = ['ss_amount','ss_delay','ll_amount','ll_delay']
pregunta.columns = ['condition'] + condition_col_name

# object containing participants' choices: '0' is choice of SS, '1' is choice of 'LL'
chavez_dt = pd.melt(chavez_dt, id_vars=['ID','School'], var_name='condition', value_name='choice')
chavez_dt = pd.merge(chavez_dt,pregunta,on='condition')
chavez_dt[condition_col_name] = chavez_dt[condition_col_name].apply(pd.to_numeric)
chavez_dt.columns = ['person_id','school','condition','choice'] + condition_col_name

In [7]:
chavez_dt

,person_id,school,condition,choice,ss_amount,ss_delay,ll_amount,ll_delay
0,1,1,DT1,0.0,54,0,55,117
1,2,1,DT1,0.0,54,0,55,117
2,3,1,DT1,0.0,54,0,55,117
3,4,1,DT1,1.0,54,0,55,117
4,5,1,DT1,0.0,54,0,55,117
...,...,...,...,...,...,...,...,...
34663,1552,8,DT27,1.0,20,0,55,7
34664,1553,8,DT27,1.0,20,0,55,7
34665,1554,8,DT27,1.0,20,0,55,7
34666,1555,8,DT27,1.0,20,0,55,7


In [11]:
data = chavez_dt
init_params = np.array([1,50,1])
dstyle = 'attention'
ustyle = 'crra'
bounds = [(0,1),(0.8,100),(0,100)]
result = minimize(mpl_estimation.log_likelihood, 
         x0=init_params, args=(data,dstyle,ustyle), 
         bounds=bounds,
         tol=1e-8,
         method='L-BFGS-B')
result

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 16993.583764312538
        x: [ 9.926e-01  8.355e-01  3.502e-01]
      nit: 13
      jac: [ 1.143e+00 -9.459e-02  1.313e-01]
     nfev: 100
     njev: 25
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

In [85]:
data = chavez_dt
init_params = np.array([.1,.1,1])
dstyle = 'quasihb'
ustyle = 'crra'
bounds = [(0,1),(0,1),(0,100)]
result = minimize(mpl_estimation.log_likelihood, 
         x0=init_params, args=(data,dstyle,ustyle), 
         bounds=bounds,
         method='L-BFGS-B')
result

     fun: 369924.0212076079
     jac: array([-1.56250000e-02, -1.21093750e-01,  1.31880775e+06])
 message: 'Optimization terminated successfully'
    nfev: 4
     nit: 5
    njev: 1
  status: 0
 success: True
       x: array([0.1, 0.1, 1. ])

In [162]:
data = chavez_dt
init_params = np.array([1,1])
dstyle = 'hb'
ustyle = 'crra'
bounds = [(0,100),(0,100)]
result = minimize(mpl_estimation.log_likelihood, 
         x0=init_params, args=(data,dstyle,ustyle), 
         bounds=bounds,
         method='L-BFGS-B')
result

      fun: 17157.30734585428
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.33905962, -0.04911271])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 123
      nit: 16
     njev: 41
   status: 0
  success: True
        x: array([0.00894893, 0.3722144 ])

In [166]:
data = chavez_dt
init_params = np.array([1,1,1])
dstyle = 'lp'
ustyle = 'crra'
bounds = [(0,1000),(0,100),(0,100)]
result = minimize(mpl_estimation.log_likelihood, 
         x0=init_params, args=(data,dstyle,ustyle), 
         bounds=bounds,
         method='L-BFGS-B')
result

      fun: 17294.079875952157
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00327418, 0.00109139, 0.01309672])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 88
      nit: 15
     njev: 22
   status: 0
  success: True
        x: array([9.88356450e+02, 3.00560959e+01, 5.16930174e-01])

In [6]:
data = chavez_dt
init_params = np.array([1,1,1])
dstyle = 'bbs'
ustyle = 'crra'
bounds = [(0,1000),(0,100),(0,100)]
result = minimize(mpl_estimation.log_likelihood, 
         x0=init_params, args=(data,dstyle,ustyle), 
         bounds=bounds,
         method='L-BFGS-B')
result

      fun: 17278.081341516026
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([89739.02404303,     0.        ,  1173.2492573 ])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 96
      nit: 9
     njev: 24
   status: 0
  success: True
        x: array([0.00892269, 0.9999999 , 0.33606459])

In [10]:
mpl_estimation.mle(data=chavez_dt,
         init_params=np.array([1,1000,1]), 
         dstyle='attention',
         ustyle='crra', 
         bounds=[(0,1),(.8,100),(0,100)])

{'params': array([0.99259818, 0.83549667, 0.35023709]),
 'se': array([0.10884479, 0.64526873, 0.45972438]),
 'log-likelihood': -16993.583764184197,
 'aic': 33993.167528368394,
 'bic': 34049.88896246992,
 'gradient': array([ 0.11677912, -0.00873115,  0.02291927])}